In [ ]:
import sys
sys.path.append('..')

from utils.DatabaseUtils import SQLLiteUtils
from utils.FileSystemUtils import FileSystemUtils
from consts.const_variables import *
from utils.common_utils import *
from utils.setup import set_me_up
from time import time
from algorithms.rock import Rock
from algorithms.clope import Clope
from algorithms.outlier_detection_algorithms import *
from algorithms.edm_cif import EDM_CIF
from utils.data_objects import *
from collections import Counter
import os
import warnings
warnings.filterwarnings("ignore")
os.chdir("..")
set_me_up()

db_session = SQLLiteUtils(database)
fs_utils = FileSystemUtils('data')
datasets, datasets_class, ohe_datasets = read_datasets_to_dict(db_session)

In [ ]:
from utils.common_utils import *
name = 'nursery_centers_1_0'
is_synthetic = 'real'
path = 'data/raw_outliers/nursery_centers_1_0.csv'
new_df = pd.read_csv(path)

class_ = new_df['class']
new_df.drop('class', axis=1, inplace=True)

new_df = (new_df.replace('?', None)
      .replace('undefined', None)
      .replace('unknown', None)
      .replace('nan', None)
      .replace(np.nan, None))
for c in new_df.columns:
    new_df[c] = new_df[c].fillna(new_df[c].mode()[0])

ohe_data = ohe(new_df)

label_codes = {label: i if label not in ['-1', -1] else -1 for i, label in enumerate(set(class_))}
coded_class = pd.Series([label_codes[i] for i in class_]).astype(int)

ohe_datasets[is_synthetic][name] = ohe_data
datasets[is_synthetic][name] = new_df.astype(str)
datasets_class[is_synthetic][name] = coded_class

In [ ]:
do_uzycia = ['agaricus-lepiota', 'balance-scale', 'car', 'house-votes-84', 'nursery', 'NPHA-doctor-visits', 'splice', 'hiv_protease_cleavage', 'Interests_group', 'bike_buyers']

do_uzycia_z_outlierami = ['primary-tumor_only_outliers']
datasets['real'].keys()

In [ ]:
################### TEST ROCK ############################

name = 'agaricus-lepiota'
dataset_type = 'real'
df = datasets[dataset_type][name]
df_ohe = ohe_datasets[dataset_type][name]
class_ = datasets_class[dataset_type][name]
print('Real classes:', len(set(class_))-1, " Real outliers:", len(class_.loc[class_==-1]))

obj = Rock(eps=0.5, k=20)
t = time()
obj.fit_predict(df.to_numpy())
t = time()-t
labels = obj.labels
print(Counter(int(x) for x in labels))

In [ ]:
# wyszukać 0.5%, 2%, 10% w rzeczywistych zbiorach
threshold = 1
result = obj.select_outliers(threshold)
print("All finded:", len(result[result==-1]), " Matches:", np.sum((result == -1) & (class_.to_numpy() == -1)))

In [ ]:
############### REAL #####################
out_process = OutlierDetectionProcess(algorithm='rock', split_mode=None, split_method=None, dataset=name, dataset_type=dataset_type,
                                      n_estimators=None, max_samples=None, min_cluster_size=None, contamination=None, max_features=None,
                                      alpha=None, beta=None, theta=None, epsilon=obj.eps, k=obj.k, ground_truth=0, clustering=1, time_sec=t,
                                      metric=None, outliers_threshold=threshold)


out_process.count_metrics(df, df_ohe, class_.to_numpy(), result)
contrastive_outlier = ContrastiveOutlier(out_process.process_id)
contrastive_outlier.calculate_configurations(df_ohe, result)

In [ ]:
out_process

In [ ]:
db_session.insert_into_table_from_dictionary(outliers_table_name, asdict(out_process))
contrastive_outlier.insert_to_db(db_session, contrastive_outlier_score_table_name)

In [ ]:
################### TEST CLOPE ############################
name = 'bike_buyers'
dataset_type = 'real'
df = datasets[dataset_type][name]
df_ohe = ohe_datasets[dataset_type][name]
class_ = datasets_class[dataset_type][name]
print('Real classes:', len(set(class_))-1, " Real outliers:", len(class_.loc[class_==-1]))

obj = Clope(r=2.3)
t = time()
obj.fit_predict(df.to_numpy())
t = time()-t
labels = obj.labels
print(Counter(int(x) for x in labels))

In [ ]:
threshold = 2
result = obj.select_outliers(threshold)
print("All finded:", len(result[result==-1])/len(df), " Matches:", np.sum((result == -1) & (class_.to_numpy() == -1)))

In [ ]:
out_process = OutlierDetectionProcess(algorithm='clope', split_mode=None, split_method=None, dataset=name, dataset_type=dataset_type,
                                      n_estimators=None, max_samples=None, min_cluster_size=None, contamination=None, max_features=None,
                                      alpha=None, beta=None, theta=obj.r, epsilon=None, k=None, ground_truth=0, clustering=1, time_sec=t,
                                      metric=None, outliers_threshold=threshold, n_neighbors=None, m=None, task=None)
out_process.count_metrics(df, df_ohe, class_.to_numpy(), result)
contrastive_outlier = ContrastiveOutlier(out_process.process_id)
contrastive_outlier.calculate_configurations(df_ohe, result)

In [ ]:
db_session.insert_into_table_from_dictionary(outliers_table_name, asdict(out_process))
contrastive_outlier.insert_to_db(db_session, contrastive_outlier_score_table_name)

In [ ]:
################### TEST ISOLATION FOREST, LOF, CBLOF, DBSCAN ############################

name = 'dataset_1'
dataset_type = 'synthetic'
df = datasets[dataset_type][name]
df_ohe = ohe_datasets[dataset_type][name]
class_ = datasets_class[dataset_type][name]
print('Real classes:', len(set(class_))-1, " Real outliers:", len(class_.loc[class_==-1]))

t = time()

# Isolation Forest algorithm
iso, iso_params, iso_labels = IsolationForest(df_ohe, 0.02)

# LOF algorithm
lof, lof_params, lof_labels = LOF(df_ohe, 0.02)

# CBLOF algorithm
cblof, cblof_params, cblof_labels = CBLOF(df_ohe, 0.02)

# HDBSCAN algorithm
hdbscan, hdbscan_params, hdbscan_labels = HDBSCAN(df_ohe)


t = time()-t
print(Counter(int(x) for x in labels))
print("All finded:", len(labels[labels==-1])/len(df), " Matches:", np.sum((labels == -1) & (class_.to_numpy() == -1)))

In [ ]:
out_process = OutlierDetectionProcess(algorithm='lof', split_mode=None, split_method=None, dataset=name, dataset_type=dataset_type,
                                      n_estimators=None, max_samples=None, min_cluster_size=None, contamination=iso.contamination, max_features=None,
                                      alpha=None, beta=None, theta=None, epsilon=None, k=None, ground_truth=0, clustering=0, time_sec=t,
                                      metric=iso.metric, outliers_threshold=None, n_neighbors=iso.n_neighbors, m=None)
out_process.count_metrics(df, df_ohe, class_.to_numpy(), labels)
contrastive_outlier = ContrastiveOutlier(out_process.process_id)
contrastive_outlier.calculate_configurations(df_ohe, labels)
db_session.insert_into_table_from_dictionary(outliers_table_name, asdict(out_process))
contrastive_outlier.insert_to_db(db_session, contrastive_outlier_score_table_name)

In [ ]:
out_process = OutlierDetectionProcess(algorithm='isolation_forest', split_mode=None, split_method=None, dataset=name, dataset_type=dataset_type,
                                      n_estimators=iso.n_estimators, max_samples=iso.max_samples, min_cluster_size=None, contamination=iso.contamination, max_features=iso.max_features,
                                      alpha=None, beta=None, theta=None, epsilon=None, k=None, ground_truth=1, clustering=0, time_sec=t,
                                      metric=None, outliers_threshold=None, n_neighbors=None, m=None)
out_process.count_metrics(df, df_ohe, class_.to_numpy(), labels)
contrastive_outlier = ContrastiveOutlier(out_process.process_id)
contrastive_outlier.calculate_configurations(df_ohe, labels)